In [22]:
# script to calculate ratios for use in TTS method 
# match rf 11 to varying BL for ratios 
# 1) RF11 UT to RF11 BL 
# 2) RF11 UT to campaign average BL 
# 3) RF11 UT to RF11, 10, 09 BL 
# author: sofia chelpon
# date created: 3-28-2020
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? 
Nothing done.


In [4]:
import numpy as np 
import xarray as xr
import pandas as pd 
import datetime 
import seaborn as sns
import datetime as dt
from scipy.optimize import least_squares

import matplotlib
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
from mpl_toolkits import mplot3d

%matplotlib inline
%config inlinebackend.figure_format='retina'

In [5]:
# load in both toga and awas data pickles 
awas_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/awas_data_df.pkl")
toga_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/toga_data_df.pkl")

In [8]:
################# -------------------- TOGA  -------------------- #################


In [17]:
# get mean in UT and BL for 1) flights in total and 2) individual flights 
toga_means = pd.DataFrame()

# campaign avg - UT 
toga_means['UT - Camp Avg'] = toga_df.loc[(toga_df['GGALT'] > 12000) 
                                      & (toga_df['GGALT'] < 14000)].mean()

# 2 km - BL 
toga_means['BL - 2 km'] = toga_df[(toga_df['GGALT'] < 2000)].mean()

# 1 km - BL
toga_means['BL - 1 km'] = toga_df.loc[(toga_df['GGALT'] < 1000)].mean()

# 0.5 km - BL
toga_means['BL - 0.5 km'] = toga_df.loc[(toga_df['GGALT'] < 500)].mean()

#toga_means.index.name = "Trace_Gas"
toga_means = toga_means.drop('GGALT').drop('GGLAT').drop('GGLON')
toga_means = toga_means.reset_index()
toga_means = toga_means.rename(columns={'index': 'Trace_Gas'})

In [16]:
# ratios for 1) flights in total and 2) individual flights 
toga_ratios = pd.DataFrame()

toga_ratios['Trace_Gas'] = toga_means['Trace_Gas']
toga_ratios['CampAvg_2kmBL'] = toga_means['UT - Camp Avg']/toga_means['BL - 2 km']
toga_ratios['CampAvg_1kmBL'] = toga_means['UT - Camp Avg']/toga_means['BL - 1 km']
toga_ratios['CampAvg_0.5kmBL'] = toga_means['UT - Camp Avg']/toga_means['BL - 0.5 km']

toga_ratios

# drop species names so doesnt duplicate column when merged 
toga_ratios = toga_ratios.drop('Trace_Gas', axis = 1)
toga_ratios

,CampAvg_2kmBL,CampAvg_1kmBL,CampAvg_0.5kmBL
0,0.095296,0.088554,0.089608
1,0.182235,0.157733,0.156980
2,0.229626,0.198960,0.198968
3,0.340642,0.296981,0.297468
4,0.408074,0.352060,0.346275
5,0.338640,0.352470,0.368870
6,0.594153,0.563845,0.548365
7,0.585199,0.547400,0.535839
8,0.943288,0.918287,0.899862
9,0.757664,0.730089,0.728671


In [18]:
### add attribute to each species (BL, tropo, and UT lifetimes)
# read in from xls file to make dataframe
mypath2 = '/Volumes/scdrive2/TTS_2020/contrast_readin/toga_lodhalf/toga_lifetimes_12162019.xlsx'
toga_lifetimes = pd.read_excel(mypath2)

# merge lifetimes to ratios
toga_ratios_full = toga_lifetimes.merge(toga_ratios, left_index=True, right_index=True)
toga_ratios_full.insert(0, 'Instrument', 'TOGA')

In [19]:
toga_ratios_full

,Instrument,Trace_Gas,BL_tau,TROPO_tau,UT_tau,CampAvg_2kmBL,CampAvg_1kmBL,CampAvg_0.5kmBL
0,TOGA,Acetaldehyde,0.289063,0.365920,0.376967,0.095296,0.088554,0.089608
1,TOGA,n_Butane,1.959376,3.813463,8.057685,0.182235,0.157733,0.156980
2,TOGA,i_Butane,2.175570,4.031311,7.903592,0.229626,0.198960,0.198968
3,TOGA,Benzene,3.755136,6.447323,11.947207,0.340642,0.296981,0.297468
4,TOGA,C3H8,4.214557,9.062693,22.450000,0.408074,0.352060,0.346275
5,TOGA,Methanol,5.160001,9.567463,20.040000,0.338640,0.352470,0.368870
6,TOGA,Bromoform,15.069605,23.166062,33.432173,0.594153,0.563845,0.548365
7,TOGA,Tetrachloroethylene,27.471720,77.860538,317.837040,0.585199,0.547400,0.535839
8,TOGA,Dibromomethane,39.129546,95.533111,306.353687,0.943288,0.918287,0.899862
9,TOGA,Dichloromethane,45.867749,109.800998,343.676023,0.757664,0.730089,0.728671


In [10]:
################# -------------------- AWAS  -------------------- #################

In [20]:
# get mean in UT and BL for 
awas_means = pd.DataFrame()

# campaign avg - UT 
awas_means['UT - Camp Avg'] = awas_df.loc[(awas_df['GGALT'] > 12000) 
                                      & (awas_df['GGALT'] < 14000)].mean()

# 2 km - BL 
awas_means['BL - 2 km'] = awas_df[(awas_df['GGALT'] < 2000)].mean()

# 1 km - BL
awas_means['BL - 1 km'] = awas_df.loc[(awas_df['GGALT'] < 1000)].mean()

# 0.5 km - BL
awas_means['BL - 0.5 km'] = awas_df.loc[(awas_df['GGALT'] < 500)].mean()

#toga_means.index.name = "Trace_Gas"
awas_means = awas_means.drop('GGALT').drop('GGLAT').drop('GGLON')
awas_means = awas_means.reset_index()
awas_means = awas_means.rename(columns={'index': 'Trace_Gas'})

In [21]:
# ratios for 1) flights in total and 2) individual flights 
awas_ratios = pd.DataFrame()

awas_ratios['Trace_Gas'] = awas_means['Trace_Gas']
awas_ratios['CampAvg_2kmBL'] = awas_means['UT - Camp Avg']/awas_means['BL - 2 km']
awas_ratios['CampAvg_1kmBL'] = awas_means['UT - Camp Avg']/awas_means['BL - 1 km']
awas_ratios['CampAvg_0.5kmBL'] = awas_means['UT - Camp Avg']/awas_means['BL - 0.5 km']

# drop species names so doesnt duplicate column when merged 
awas_ratios = awas_ratios.drop('Trace_Gas', axis = 1)

In [22]:
### add attribute to each species (BL, tropo, and UT lifetimes)
# read in from xls file to make dataframe
mypath2 = '/Volumes/scdrive2/TTS_2020/contrast_readin/awas/awas_lifetimes_12162019.xlsx'
awas_lifetimes = pd.read_excel(mypath2)
len(awas_lifetimes)

# merge lifetimes to ratios 
awas_ratios_full = awas_lifetimes.merge(awas_ratios, left_index=True, right_index=True)
awas_ratios_full.insert(0, 'Instrument', 'AWAS')

In [23]:
awas_ratios_full

,Instrument,Trace_Gas,BL_tau,TROPO_tau,UT_tau,CampAvg_2kmBL,CampAvg_1kmBL,CampAvg_0.5kmBL
0,AWAS,C2H6_Ethane,18.555721,48.949092,167.260709,0.583969,0.563214,0.549628
1,AWAS,C2H2_Ethyne,5.422335,9.603369,18.712675,0.460012,0.443029,0.431870
2,AWAS,C3H8_Propane,4.214557,9.062693,22.450000,0.266881,0.254867,0.246854
3,AWAS,C4H10_Isobutane,2.175570,4.031311,7.903592,0.191701,0.179751,0.174816
4,AWAS,C4H10_nButane,1.959376,3.813463,8.057685,0.198149,0.189100,0.184079
5,AWAS,C5H12_nPentane,1.214449,2.312606,4.723079,0.186828,0.186275,0.180812
6,AWAS,C6H6_Benzene,3.755136,6.447323,11.947207,0.358971,0.336431,0.330359
7,AWAS,OCS_Carbonyl_sulfide,2000.000000,2000.000000,2000.000000,1.004043,1.005519,1.003988
8,AWAS,C2H6S_Dimethyl_sulfide,0.955122,1.689426,3.279163,0.019572,0.016854,0.015182
9,AWAS,CCl2F2_CFC_12,37000.000000,37000.000000,37000.000000,1.003408,1.002893,1.002589


In [15]:
################# -------------------- MERGE THE TWO  -------------------- #################

In [24]:
master_list = awas_ratios_full
master_list = master_list.append(toga_ratios_full, ignore_index = True, sort = False)

# pickle this!! 
master_list.to_pickle("./contrast_ratios_varyBLheight.pkl")

# look at output 
#master_list[master_list['Trace_Gas'] == 'Dichloromethane'].drop('BL_tau', axis = 1).drop('UT_tau', axis = 1)

In [25]:
master_list

,Instrument,Trace_Gas,BL_tau,TROPO_tau,UT_tau,CampAvg_2kmBL,CampAvg_1kmBL,CampAvg_0.5kmBL
0,AWAS,C2H6_Ethane,18.555721,48.949092,167.260709,0.583969,0.563214,0.549628
1,AWAS,C2H2_Ethyne,5.422335,9.603369,18.712675,0.460012,0.443029,0.431870
2,AWAS,C3H8_Propane,4.214557,9.062693,22.450000,0.266881,0.254867,0.246854
3,AWAS,C4H10_Isobutane,2.175570,4.031311,7.903592,0.191701,0.179751,0.174816
4,AWAS,C4H10_nButane,1.959376,3.813463,8.057685,0.198149,0.189100,0.184079
5,AWAS,C5H12_nPentane,1.214449,2.312606,4.723079,0.186828,0.186275,0.180812
6,AWAS,C6H6_Benzene,3.755136,6.447323,11.947207,0.358971,0.336431,0.330359
7,AWAS,OCS_Carbonyl_sulfide,2000.000000,2000.000000,2000.000000,1.004043,1.005519,1.003988
8,AWAS,C2H6S_Dimethyl_sulfide,0.955122,1.689426,3.279163,0.019572,0.016854,0.015182
9,AWAS,CCl2F2_CFC_12,37000.000000,37000.000000,37000.000000,1.003408,1.002893,1.002589
